In [66]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Embedding, GRU, Dense
from tensorflow.keras.optimizers import SGD

In [64]:
np.random.seed(42)
tf.random.set_seed(42)

Finte State Machine for Reber Grammar   
   
![Reber Grammar](Images/reber.gif "Reber Grammar")

In [3]:
reber_grammar = [
    [("B", 1)],           
    [("T", 2), ("P", 3)], 
    [("S", 2), ("X", 4)], 
    [("T", 3), ("V", 5)], 
    [("X", 3), ("S", 6)],
    [("P", 4), ("V", 6)],
    [("E", None)]]        

Finite State Machine for Embedded Reber Grammar   
   
![Embedded Reber Grammar](Images/emb_reber.gif "Embedded Reber Grammar")

In [6]:
embedded_reber_grammar = [
    [("B", 1)],
    [("T", 2), ("P", 3)],
    [(reber_grammar, 4)],
    [(reber_grammar, 5)],
    [("T", 6)],
    [("P", 6)],
    [("E", None)]]

In [29]:
# This function generates strings based on the (Embedded) Reber Grammar
def generate(grammar):
    state, output = 0, []
    while state is not None:
        index = np.random.randint(len(grammar[state]))
        production, state = grammar[state][index]
        if isinstance(production, list):
            production = generate(grammar=production)
        output.append(production)
    return "".join(output)

In [61]:
for _ in range(30):
    print(generate(reber_grammar), end=" ")

BTXXTTVPXTVPXTTVPSE BPVPSE BTXSE BPVVE BPVVE BTSXSE BPTVPXTTTVVE BPVVE BTXSE BTXXVPSE BPTTTTTTTTVVE BTXSE BPVPSE BTXSE BPTVPSE BTXXTVPSE BPVVE BPVVE BPVVE BPTTVVE BPVVE BPVVE BTXXVVE BTXXVVE BTXXVPXVVE BTXSE BPTTTTTTTTTVPSE BPVPXTTVPXTTTVPSE BTXXTVVE BPTVPXVVE 

In [42]:
for _ in range(20):
    print(generate(embedded_reber_grammar), end=" ")

BPBPVVEPE BPBPTVPSEPE BPBTXXVVEPE BTBPTVPXVVETE BTBPVVETE BTBTSSSSSSSXXVVETE BPBTSSSXXTTTTVPSEPE BTBPTTVVETE BPBTXXTVVEPE BTBTXSETE BPBTSSSSSXXTTVPXVPXTTTVVEPE BPBTSSXXTVPSEPE BPBPTTTTTTTVPSEPE BTBTSXSETE BPBPTVPXVVEPE BPBPVVEPE BPBPTVVEPE BTBPTTVPXTTVPSETE BTBTSSXSETE BTBTXXTTVVETE 

In [34]:
POSSIBLE_CHARS = "BEPSTVX"

# This function generate strings that do not respect the grammar
# We generate a string that respects the grammar, and we will corrupt it by changing just one character
def generate_corrupted(grammar, chars=POSSIBLE_CHARS):
    good_string = generate(grammar)
    index = np.random.randint(len(good_string))
    good_char = good_string[index]
    bad_char = np.random.choice(sorted(set(chars) - set(good_char)))
    return good_string[:index] + bad_char + good_string[index + 1:]

In [44]:
for _ in range(20):
    print(generate_corrupted(embedded_reber_grammar), end=" ")

BTBTXPETE BTBTSSSXXTTVSXVPXTTTVVETE BTBXTTVPSETE BEBTSSSSSXXVPXTVVETE BTBXTTVVETE BPBTXSTPE BTBTXXTTTVPSBTE BTBTXSETX BTBTSXSSTE BPBPVVEPT BTBPTVEETE BTBTSSXXTTVXETE BTBTSXTTVVETE BPBPVVTPE BTBTSXTTVVETE EPBPVPXVVEPE BPTTXSEPE BPBTXXSPXTVVEPE BTBTXSPTE BPTTSXXTVPXVVEPE 

In [45]:
# Function to embed input strings
def str_to_ids(s, chars=POSSIBLE_CHARS):
    return [chars.index(c) for c in s]

In [51]:
# Function to generate 50% good and 50% bad strings
def generate_dataset(size):
    good_strings = [string_to_ids(generate(embedded_reber_grammar))
                    for _ in range(size // 2)]
    bad_strings = [string_to_ids(generate_corrupted(embedded_reber_grammar))
                   for _ in range(size - size // 2)]
    all_strings = good_strings + bad_strings
    X = tf.ragged.constant(all_strings, ragged_rank=1)
    y = np.array([[1.] for _ in range(len(good_strings))] +
                 [[0.] for _ in range(len(bad_strings))])
    return X, y

In [55]:
X_train, y_train = generate_dataset(10000)
X_valid, y_valid = generate_dataset(2000)

In [63]:
X_train[0], y_train[0]

(<tf.Tensor: shape=(16,), dtype=int32, numpy=array([0, 4, 0, 4, 3, 3, 6, 6, 5, 2, 6, 5, 5, 1, 4, 1], dtype=int32)>,
 array([1.]))

In [67]:
embedding_size = 5

model = Sequential([
    InputLayer(input_shape=[None], dtype=tf.int32, ragged=True),
    Embedding(input_dim=len(POSSIBLE_CHARS), output_dim=embedding_size),
    GRU(30),
    Dense(1, activation="sigmoid")
])

optimizer = SGD(lr=0.02, momentum=0.95, nesterov=True)
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

Epoch 1/20
/home/prateekd007/.local/lib/python3.8/site-packages/tensorflow/python/framework/indexed_slices.py:435: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential/gru/RaggedToTensor/boolean_mask_1/GatherV2:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential/gru/RaggedToTensor/boolean_mask/GatherV2:0", shape=(None, 5), dtype=float32), dense_shape=Tensor("gradient_tape/sequential/gru/RaggedToTensor/Shape:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
313/313 [==============================] - 6s 14ms/step - loss: 0.6931 - accuracy: 0.5033 - val_loss: 0.6794 - val_accuracy: 0.5690
Epoch 2/20
313/313 [==============================] - 4s 13ms/step - loss: 0.6725 - accuracy: 0.5553 - val_loss: 0.6510 - val_accuracy: 0.6205
Epoch 3/20
313/313 [==============================] - 4s 13ms/step - loss: 0.6514 - accuracy: 0.5797 - val_loss: 0.6408

In [69]:
test_strings = [
    "BPBTSSSSSSSXXTTVPXVPXTTTTTVVETE",
    "BPBTSSSSSSSXXTTVPXVPXTTTTTVVEPE"
]
X_test = tf.ragged.constant([string_to_ids(s) for s in test_strings], ragged_rank=1)

y_proba = model.predict(X_test)
print("Estimated probability that these are Reber strings:")
for index, string in enumerate(test_strings):
    print("{}: {:.2f}%".format(string, 100 * y_proba[index][0]))

Estimated probability that these are Reber strings:
BPBTSSSSSSSXXTTVPXVPXTTTTTVVETE: 1.77%
BPBTSSSSSSSXXTTVPXVPXTTTTTVVEPE: 99.95%


Our RNN identifies the pattern  of Embedded Reber Grammar i.e. the second letter should always be equal to the second to last letter with  very high proabability. That requires a fairly long short-term memory.